In [ ]:
# Install Dependencies
!pip install transformers datasets torch scikit-learn

In [ ]:
# Load and Preprocess Data
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("wangyuancheng/discord-phishing-scam")
ds = ds.rename_column('lable', 'label') # Renaming 'lable' column to 'label'

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

(…)iscord-message-phishing%20-%20Sheet1.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# View Dataset
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'msg_content', 'msg_timestamp', 'usr_joined_at', 'time_since_join', 'message_length', 'word_count', 'has_link', 'has_mention', 'num_roles', 'Unnamed: 10', 'Unnamed: 11'],
        num_rows: 2000
    })
})

In [ ]:
# Keep only the text and label columns
ds = ds.remove_columns([
    col for col in ds["train"].column_names
    if col not in ["msg_content", "label"]
])

ds

DatasetDict({
    train: Dataset({
        features: ['label', 'msg_content'],
        num_rows: 2000
    })
})

In [ ]:
# Check for missing values
train_df = ds["train"].to_pandas()
print(train_df.isna().sum())

label          0
msg_content    0
dtype: int64


In [ ]:
# Train and validate our data
from datasets import ClassLabel, Features

# Assuming the labels are 0 and 1, you can define the ClassLabel feature
# If you have different labels, adjust the names accordingly.
features = ds["train"].features
features["label"] = ClassLabel(num_classes=2, names=[0, 1])
ds = ds.cast(features)

ds = ds["train"].train_test_split(
    test_size=0.1,
    stratify_by_column="label"
)

ds["train"], ds["validation"] = ds["train"], ds["test"]
del ds["test"]

ds

Casting the dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'msg_content'],
        num_rows: 1800
    })
    validation: Dataset({
        features: ['label', 'msg_content'],
        num_rows: 200
    })
})

In [ ]:
# Tokenizer
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch["msg_content"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Preprocess
tokenized_ds = ds.map(tokenize, batched=True)

# Renaming label and convert to Pytorch sensors
tokenized_ds = tokenized_ds.rename_column("label", "labels")

tokenized_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"],
)

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# Verify tokenized data structure
print("Tokenized dataset structure:")
print(tokenized_ds)

Tokenized dataset structure:
DatasetDict({
    train: Dataset({
        features: ['labels', 'msg_content', 'input_ids', 'attention_mask'],
        num_rows: 1800
    })
    validation: Dataset({
        features: ['labels', 'msg_content', 'input_ids', 'attention_mask'],
        num_rows: 200
    })
})


In [ ]:
# Load Model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=ds['train'].features['label'].num_classes
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define evaluation metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary'
    )
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device:", device)

using device: cuda


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/sentry-checkpoints",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=30,
    fp16=True,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    compute_metrics=compute_metrics
)

# Start training
print("Starting training...")
train_result = trainer.train()

# Print training metrics
print("\nTraining completed!")
print(f"Training loss: {train_result.training_loss:.4f}")
print(f"Training time: {train_result.metrics['train_runtime']:.2f} seconds")

Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.083900,0.187215,0.945000,0.819672,0.757576,0.892857
2,0.025000,0.297411,0.940000,0.785714,0.785714,0.785714
3,0.012700,0.321296,0.940000,0.793103,0.766667,0.821429
4,0.011400,0.342735,0.950000,0.821429,0.821429,0.821429
5,0.007700,0.334588,0.945000,0.807018,0.793103,0.821429
6,0.000400,0.349368,0.935000,0.771930,0.758621,0.785714
7,0.000200,0.410216,0.945000,0.800000,0.814815,0.785714
8,0.000100,0.348157,0.945000,0.807018,0.793103,0.821429
9,0.000200,0.526297,0.935000,0.754717,0.800000,0.714286
10,0.000100,0.406122,0.950000,0.821429,0.821429,0.821429



Training completed!
Training loss: 0.0045
Training time: 526.66 seconds


In [ ]:
# Evaluate the model on validation set
print("Evaluating model...")
eval_results = trainer.evaluate()

print("\nValidation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

Evaluating model...



Validation Results:
eval_loss: 0.3427
eval_accuracy: 0.9500
eval_f1: 0.8214
eval_precision: 0.8214
eval_recall: 0.8214
eval_runtime: 0.2297
eval_samples_per_second: 870.7410
eval_steps_per_second: 30.4760
epoch: 30.0000


In [ ]:
# Test the model with sample predictions
test_messages = [
    "50$ gift - [steamcommunity.com/gift/activation/id=576088601105](https://tinyurl.com/3a9er9hx) @everyone",
    "Hey when did you join the server?",
    "URGENT: Your account will be suspended unless you verify immediately with your mobile number and code",
    "Did you guys download Battlefield 6 yet?"
]

print("Testing model predictions:\n")
for msg in test_messages:
    inputs = tokenizer(msg, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()} # Move inputs to the same device as the model
    outputs = model(**inputs)
    prediction = outputs.logits.argmax(-1).item()
    confidence = outputs.logits.softmax(-1).max().item()

    label = "SPAM" if prediction == 1 else "HAM"
    print(f"Message: {msg}")
    print(f"Prediction: {label} (confidence: {confidence:.2%})\n")

Testing model predictions:

Message: 50$ gift - [steamcommunity.com/gift/activation/id=576088601105](https://tinyurl.com/3a9er9hx) @everyone
Prediction: SPAM (confidence: 99.84%)

Message: Hey when did you join the server?
Prediction: HAM (confidence: 99.99%)

Message: URGENT: Your account will be suspended unless you verify immediately with your mobile number and code
Prediction: SPAM (confidence: 99.60%)

Message: Did you guys download Battlefield 6 yet?
Prediction: HAM (confidence: 99.99%)



In [ ]:
# Save the trained model
model_save_path = "./discord_scam_model"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

Model and tokenizer saved to ./phishing_detection_model


In [ ]:
import shutil
from google.colab import files

# Path to the folder you want to download
folder_to_download = "./discord_scam_model"

# Create a zip archive of the folder
zip_filename = "discord_scam_model.zip"
shutil.make_archive(folder_to_download, 'zip', folder_to_download)

# Download the zip file
files.download(zip_filename)

print(f"'{folder_to_download}' has been zipped to '{zip_filename}' and downloaded.")